In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250
n_embd = 384

cuda


In [20]:
with open('wiz_of_oz.txt','r', encoding = 'utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(len(chars))

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [21]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello = encode("Hello")

print(encoded_hello)

decoded_hello = decode(encoded_hello)
print(decoded_hello)

[32, 58, 65, 65, 68]
Hello


In [22]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:200])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50,  9,  1, 39, 50, 37, 25,  1, 39,
        30,  1, 39, 50,  9,  1, 29, 44, 27, 11,  0,  0,  1,  1, 33, 36, 36, 45,
        43, 44, 42, 25, 44, 29, 28,  1, 26, 49,  1, 34, 39, 32, 38,  1, 42, 11,
         1, 38, 29, 33, 36, 36,  0,  0,  1,  1, 26, 39, 39, 35, 43,  1, 39, 30,
         1, 47, 39, 38, 28, 29, 42,  1, 47, 33, 36, 36, 33, 25, 37,  1, 37, 39,
        42, 42, 39, 47,  1,  4,  1, 27, 39, 11,  9,  1, 33, 38, 27, 11,  1, 38,
        29, 47])


In [23]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return(x, y)
x,y = get_batch('train')
print('inputs:')
print(x)
print('outputs:')
print(y)

tensor([185298,  19490,   1709, 121155])
inputs:
tensor([[ 1, 62, 73,  1, 76, 68, 74, 65],
        [67, 68,  1, 57, 54, 67, 60, 58],
        [67, 58, 57,  1, 73, 68,  1, 73],
        [73,  1, 73, 61, 58, 62, 71,  1]], device='cuda:0')
outputs:
tensor([[62, 73,  1, 76, 68, 74, 65, 57],
        [68,  1, 57, 54, 67, 60, 58, 71],
        [58, 57,  1, 73, 68,  1, 73, 61],
        [ 1, 73, 61, 58, 62, 71,  1, 58]], device='cuda:0')


In [24]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [16]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.posiion_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


4H!y[BV2W8P UQ3jae-'?pQSFIVz_py7U('DU"u&Hns!4([]ZDk,VJ.z;oL.hV';doLME"EX]JMI12)ird([x9,UFf g_k﻿is )l,eC.I1QpvpSPX99GcX!?ik(aV:tlsaVpE9tB[B4TaqJ[Vsxwj'i0pdvHaf4TKB3NQuSpUW7L_"aHTB9V(E. 
8dOptHFgE5(:uTN7b7lg;('0g.R]Q-Rl296_tmU;i;hSwjhDv:ceq"M)eCLHP.By9n!4!﻿"CK3)s89LAd,wFjSmv0wAb08VZWSUEFYOHtcIx!ON)3jSAL"A!.z&;PhQV*;o,2WiR_MyCS*xW-CjF.g5BSFJ﻿zjWk,obBTYz﻿y7YDjwbX_DCKj"E2Pv :3qZDKz.'KoRKtEO)2?neuUbF SAtH
E,'XX9TjS*TjP.oVNs *EgMF0,J0Pf(-pR('q﻿8&Xlysnhk*fuw?U)y&Zuck6
XbE8PuSVN&mN"Pba15,0BLZ._"VzV*xWrr_


In [17]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

tensor([ 9231, 29113, 71450, 97352])
tensor([104589,  93593,  67400,  22427])
tensor([ 38997,  93607,   6294, 162619])
tensor([25756, 83760,  6936, 59063])
tensor([ 82228, 182414, 158565,  56715])
tensor([ 24592, 184711, 113064, 164833])
tensor([110166, 148284, 178199,  89606])
tensor([ 33867, 167534,  10790, 176924])
tensor([163188,  88071,  32389,  38544])
tensor([125315, 156535,  52671,   3070])
tensor([140354,  26838,  24725, 110773])
tensor([ 44827, 123463,  53534, 141539])
tensor([49267,  2102, 74716, 44784])
tensor([173346,  77700,  33827, 166062])
tensor([168446,   9401, 167139,  57597])
tensor([179522,  53585, 184739,  71542])
tensor([104480,  40861,  59610,  36178])
tensor([110401, 105960,  36802,  93581])
tensor([ 71242,  78165, 169764, 173534])
tensor([153312,   1267,  88749, 163049])
tensor([157795, 112509, 100069, 180468])
tensor([142536, 165377,  80655,  98701])
tensor([ 13092, 102827, 124284, 112210])
tensor([ 45155, 183558, 176698, 103029])
tensor([101245,  12126, 1028

In [18]:
context = torch.zeros((1,1), dtype = torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())  
print(generated_chars)


'0ox(AVJQgwXxa﻿5T]N?9!SfS]ZGOoMC"R6P!﻿8d3hV'FNzSkG)WHRv!h)vBWc
LE1GY'Xh)4T:DHfVC67G)4;tA'oWv-D﻿Y:w,8YQf99!Df4BuXJw4R4x[9g4!oQ:t6AhVSDptUOPuS"Lp6fbIr9,fbokA;r7DRZGrM91'n-,Axn0YwA(euDi?Ba_rB?4e-yX9ZDfUit:)V  r'BqscgaSV*l N[Bc ?PqtO;D!gZNzHMFCzyb﻿xSUZ&kI[8&z3z-ueCP2XTaXk,f I61f"nJ'h2W._﻿.UJNg.'Z6FG8
(iuDjUFu*[4] Z18z.'*AtOUwZyGR8jS(3a_w5.;BZGn&;f8rre.ZEDj
0r-RW'*ADjg5AuSUspamzSkuJQgcmrpEeCOBS;r'X
nkc1']j'-(6Opl[B,dBqqZR;!0-PsCzSP
bc-X1'f kv:)eN &﻿R?Zv1:tDf-)!?oZvDHyUThHW*FjV.7gdO;GaSEjMKr7.)XiJ'-pE
